In [17]:
!pip install python-dotenv
!pip install snowflake-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 30.9 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.0
    Uninstalling cryptography-43.0.0:
      Successfully uninstalled cryptography-43.0.0


In [16]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the variables
user = os.getenv('USER_NAME')
password = os.getenv('PASSWORD')
account = os.getenv('ACCOUNT')
aws_key_id = os.getenv('AWS_KEY_ID')
aws_secret_key = os.getenv('AWS_SECRET_KEY')
#.env
# USER_NAME=
# PASSWORD=
# ACCOUNT=
# AWS_KEY_ID= 
# AWS_SECRET_KEY= 

In [18]:
import snowflake.connector
conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse='compute_wh',
    database='TEST',
    schema='TEST'
)
cursor = conn.cursor()


In [19]:
cursor.execute("""
select *, row_number() over(partition by state order by total_population) as rank
from COVID19_EPIDEMIOLOGICAL_DATA.PUBLIC.DEMOGRAPHICS order by state,total_population
""")
result = cursor.fetchall()
len(result)

3140

In [21]:
# python: s3 to snowflake data ingestion

cursor.execute("""
CREATE OR REPLACE FILE FORMAT my_csv_format
    TYPE = 'CSV'
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    FIELD_DELIMITER = ','
    SKIP_HEADER = 0; 
""")
result = cursor.fetchall()
result

[('File format MY_CSV_FORMAT successfully created.',)]

In [22]:
cursor.execute(f"""
CREATE OR REPLACE STAGE my_s3_stage
  URL='s3://ec2-spark-aws-files/'
  CREDENTIALS=(AWS_KEY_ID='{aws_key_id}' AWS_SECRET_KEY='{aws_secret_key}')
  FILE_FORMAT = my_csv_format;
""")
result = cursor.fetchall()
result

[('Stage area MY_S3_STAGE successfully created.',)]

In [23]:
cursor.execute("""
COPY INTO TEST.TEST.CREDIT_CARD
FROM @my_s3_stage/creditcard.csv
FILE_FORMAT = my_csv_format;
""")
result = cursor.fetchall()
result

[('Copy executed with 0 files processed.',)]

In [25]:
cursor.execute("""
select count(*) from TEST.TEST.CREDIT_CARD
""")
result = cursor.fetchall()
result

[(284807,)]